In [1]:
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")

In [2]:
import torch
import os
print("Cuda is available : ", torch.cuda.is_available())
print("Cuda num : ", torch.cuda.device_count())  # gpu数量
print("Current cuda index : ", torch.cuda.current_device())
print("Current cuda name : ", torch.cuda.get_device_name(0))

Cuda is available :  True
Cuda num :  1
Current cuda index :  0
Current cuda name :  NVIDIA GeForce RTX 3090


In [3]:
from libs import *
dataPath = "../StockPriceForecast/dataSet/data/marketData/data.nc"
data_len = DataLength(dataPath)
print("All days : ", data_len)
input_days = 360
predict_days = 5
test_predict_len = 360  # 预测360天
train_dataSet = DataSet(dataPath=dataPath, isel=[1500, data_len - input_days - test_predict_len],
                        targetDays=predict_days,
                        encoderDecoderParameter="close",
                        targetDataParameter="close")
validation_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + 2 * test_predict_len), data_len - test_predict_len],
                             targetDays=predict_days,
                             encoderDecoderParameter="close",
                             targetDataParameter="close")
test_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + test_predict_len), data_len],
                       targetDays=predict_days,
                       encoderDecoderParameter="close",
                       targetDataParameter="close")
print("Validation days ; ", validation_dataSet.data.shape)
print("Test days : ", test_dataSet.data.shape)

train_dataLoader = DataLoader(train_dataSet)
validation_dataLoader = DataLoader(validation_dataSet)
test_dataLoader = DataLoader(test_dataSet)

All days :  2994
Validation days ;  (720, 1440, 10)
Test days :  (720, 1440, 10)


In [10]:
from libs import *
from libs.modules import *
import torch
import torch.nn as nn
model = torch.load("../StockPriceForecast/Models/spf_softDTW_2_410.pth")

In [5]:
validation_dataLoader.reset()
predicts, targets, attention_weights = eval_Seq2SeqAttention(model=model,
                                                    dataLoader=validation_dataLoader,
                                                    steps=5,
                                                    device=device)

100%|██████████| 356/356 [02:15<00:00,  2.63it/s]


In [6]:
print("predicts : ", predicts.shape)
print("targets : ", targets.shape)

predicts :  torch.Size([356, 5, 1440])
targets :  torch.Size([356, 5, 1440])


In [11]:
show_batch = 6
stock_index = 7
drawTargetAndPredicts(predicts, targets, show_batch, stock_index)

In [12]:
drawTargetsAndPredicts(predicts, targets, shifter=len(train_dataLoader), stock_index=882)